In [20]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier(["I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",])

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0


[{'label': 'POSITIVE', 'score': 0.9598049521446228},
 {'label': 'NEGATIVE', 'score': 0.9966409206390381}]

In [21]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [22]:
raw_inputs = [
    "is the",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[ 101, 2003, 1996,  102]]), 'attention_mask': tensor([[1, 1, 1, 1]])}


In [23]:
## output is a dictionary that contains two keys input_id and attention_mask input_ids contains two rows of integers (one for each sentence) that are the unique identifiers of the tokens in each sentence.

101   → [CLS]
1045  → I
1005  → '
2310  → ve
2042  → been
3403  → waiting
2005  → for
1037  → a
17662 → Hugging
12172 → Face
2607  → course
2026  → my
2878  → whole
2166  → life
1012  → .
102   → [SEP]


Words can be split into subwords (Hugging + Face)

Apostrophes are handled as separate tokens

In [24]:
# lets download our pretrained model using automodel class which has from_pretrained() method

from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"

model = AutoModel.from_pretrained(checkpoint)

For each model input, we’ll retrieve a high-dimensional vector representing the contextual understanding of that input by the Transformer model.

This models head is for this task for different tasks we use different model heads.


In [25]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

torch.Size([1, 4, 768])


Outputs of transformers models behave like namedtuples or dictionaries.
To access them use outputs["last_hidden_state"]
or even by index if you know exactly where the thing you are looking for is (outputs[0]).

Lets use a model for sequence classification task i.e. to classify sentences as positive or negative

In [26]:
from transformers import AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
output = model(**inputs)

In [27]:
print(output.logits.shape)

torch.Size([1, 2])


In [28]:
print(outputs.last_hidden_state.shape)

torch.Size([1, 4, 768])


In [29]:
print(outputs.logits)

AttributeError: 'BaseModelOutput' object has no attribute 'logits'

In [30]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

AttributeError: 'BaseModelOutput' object has no attribute 'logits'

In [31]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}